# DSC 232R: Foundations & Applications of PySpark RDDs

This notebook covers the basics of Spark and more complex data processing patterns.

### Learning Objectives:
1. **Environment**: Setting up Spark on Google Colab.
2. **Basics**: Creating RDDs and understanding Lazy Evaluation.
3. **Transformations**: Deep dive into `map`, `filter`, and `flatMap`.
4. **Actions**: Triggering the computation graph.
5. **Pair RDDs**: Handling Key-Value data (Aggregations & Joins).
6. **Advanced**: Set operations, Caching, and Real-world Log Analysis.

## 1. Installation and SparkContext
Spark runs on the JVM, so we must install Java 8 and the Spark binaries first.

In [1]:
# Install Java and Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("DSC232R_Discussion").setMaster("local[*]")
sc = SparkContext(conf=conf)
print("Spark Context Initialized!")

Spark Context Initialized!


## 2. Creating RDDs (The Basics)
There are two primary ways to get data into Spark.

### What is an RDD?
* **Resilient**: Fault-tolerant, able to recompute missing partitions.
* **Distributed**: Data is partitioned across multiple nodes in a cluster.
* **Dataset**: A collection of objects.

### The Spark Architecture

When you execute code, this is how Spark distributes the work:

```text
       [ Driver Program ]
              | (SparkContext)
     --------------------------
     |            |           |
 [ Worker ]   [ Worker ]  [ Worker ]
 (Executor)   (Executor)  (Executor)
    /   \       /   \       /   \
 [Task][Task] [Task][Task] [Task][Task]
```

**Transformations** build the plan (DAG). **Actions** trigger the execution.

In [2]:
# Method 1: Parallelizing an existing Python collection
data_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
rdd_from_list = sc.parallelize(data_list)
print("RDD from List:", rdd_from_list.collect())

RDD from List: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [3]:
# Method 2: Reading from an external file
# Step A: We generate 'numbers.txt' locally first so this code is self-contained
with open("numbers.txt", "w") as f:
    f.write("10\n20\n30\n40\n50")

# Step B: Load the file into an RDD
file_rdd = sc.textFile("numbers.txt")
print("Raw RDD (Strings):", file_rdd.collect())

# Step C: Convert strings to integers for calculation
int_rdd = file_rdd.map(lambda x: int(x))
print("Integer RDD:", int_rdd.collect())

Raw RDD (Strings): ['10', '20', '30', '40', '50']
Integer RDD: [10, 20, 30, 40, 50]


## 3. Transformations: The Step-by-Step Pipeline

### Concept: Lazy Evaluation
Transformations do **not** execute immediately. They only build a recipe (the DAG).

Let's use the **Bookstore Example**: we want to find books that cost more than $40 after a 10% tax.

In [4]:
books_data = [
    ("Spark Basics", 35.00),
    ("Python for Data Science", 45.50),
    ("Distributed Systems", 60.00),
    ("Big Data 101", 25.00)
]
books_rdd = sc.parallelize(books_data)

# Step 1: MAP - Apply 10% Tax
taxed_rdd = books_rdd.map(lambda x: (x[0], round(x[1] * 1.1, 2)))

# Show intermediate output to students
print("1. Books with Tax:", taxed_rdd.collect())

1. Books with Tax: [('Spark Basics', 38.5), ('Python for Data Science', 50.05), ('Distributed Systems', 66.0), ('Big Data 101', 27.5)]


In [5]:
# Step 2: FILTER - Filter out books <= $40
expensive_books_rdd = taxed_rdd.filter(lambda x: x[1] > 40.00)

# Show final transformed output
print("2. Expensive Books Only:", expensive_books_rdd.collect())

2. Expensive Books Only: [('Python for Data Science', 50.05), ('Distributed Systems', 66.0)]


### FlatMap vs. Map
`map` keeps the 1-to-1 relationship. `flatMap` "flattens" the structure (1-to-many).

In [6]:
sentences = ["Hello Spark", "Goodbye Hadoop"]
sentences_rdd = sc.parallelize(sentences)

map_res = sentences_rdd.map(lambda x: x.split(" ")).collect()
flat_res = sentences_rdd.flatMap(lambda x: x.split(" ")).collect()

print("Map (nested list):", map_res)
print("FlatMap (flat list):", flat_res)

Map (nested list): [['Hello', 'Spark'], ['Goodbye', 'Hadoop']]
FlatMap (flat list): ['Hello', 'Spark', 'Goodbye', 'Hadoop']


## 4. RDD Actions
Actions trigger the actual computation. Until you call these, Spark has done zero work on the data.

In [7]:
test_rdd = sc.parallelize([10, 20, 30, 40, 50])

print("Count:", test_rdd.count())
print("First item:", test_rdd.first())
print("Take 2 items:", test_rdd.take(2))
print("Sum (using Reduce):", test_rdd.reduce(lambda a, b: a + b))

Count: 5
First item: 10
Take 2 items: [10, 20]
Sum (using Reduce): 150


## 5. Pair RDDs: Working with Keys
This is where Spark becomes powerful for Big Data aggregations.

In [8]:
# Sales Data: (Author, Transaction Amount)
author_sales = [
    ("John Doe", 75), ("Jane Smith", 100),
    ("John Doe", 45), ("Bob Johnson", 50),
    ("Jane Smith", 25)
]
sales_rdd = sc.parallelize(author_sales)

# reduceByKey: Sum the sales for each author
author_totals = sales_rdd.reduceByKey(lambda a, b: a + b)
print("Total Sales per Author:", author_totals.collect())

Total Sales per Author: [('Jane Smith', 125), ('John Doe', 120), ('Bob Johnson', 50)]


In [9]:
# Join: Adding Author Ratings to the Sales Totals
author_ratings = sc.parallelize([
    ("John Doe", 4.5), ("Jane Smith", 4.8), ("Bob Johnson", 4.2)
])

joined_rdd = author_totals.join(author_ratings)
print("Joined (Author, (Total Sales, Rating)):", joined_rdd.collect())

Joined (Author, (Total Sales, Rating)): [('Jane Smith', (125, 4.8)), ('John Doe', (120, 4.5)), ('Bob Johnson', (50, 4.2))]


## 6. Advanced Concepts

### Set Operations and Caching
Set operations like `union` and `intersection` are useful for user-group comparisons.

In [10]:
group_a = sc.parallelize(["User1", "User2", "User3"])
group_b = sc.parallelize(["User3", "User4", "User5"])

print("Union (All unique users):", group_a.union(group_b).distinct().collect())
print("Intersection (Common users):", group_a.intersection(group_b).collect())

# CACHING
# If we use an RDD multiple times, we cache it in memory for performance.
group_a.cache()
print("Group A is now cached.")

Union (All unique users): ['User2', 'User3', 'User5', 'User1', 'User4']
Intersection (Common users): ['User3']
Group A is now cached.


### Practical Exercise: Production-Style Log Analysis
Given raw logs, we want to find the most frequent ERROR message.

In [11]:
raw_logs = [
    "[ERROR] - Timeout", "[INFO] - Boot", "[ERROR] - Timeout",
    "[ERROR] - DB Fail", "[WARNING] - High Mem", "[ERROR] - Timeout"
]
logs_rdd = sc.parallelize(raw_logs)

# The Pipeline:
# 1. Filter only Errors
# 2. Extract the message string
# 3. Pair with a count of 1
# 4. Sum counts by message
# 5. Sort by frequency

result = logs_rdd \
    .filter(lambda line: "[ERROR]" in line) \
    .map(lambda line: (line.split(" - ")[1], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda x: x[1], ascending=False)

print("Log Analysis Result:", result.collect())

Log Analysis Result: [('Timeout', 3), ('DB Fail', 1)]


## Diagram: Understanding the DAG and Lineage

```text
[ Parallelize ] -> RDD_A
     |
     v
[ map(tax) ]   -> RDD_B (Depends on RDD_A)
     |
     v
[ filter(40) ] -> RDD_C (Depends on RDD_B)
     |
     v
[ collect() ]  <- ACTION (Triggers re-calculation of A -> B -> C)
```

**Fault Tolerance**: If RDD_B is lost, Spark uses the graph above to recompute it from RDD_A!